# import Libraries

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import pandas as pd 
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from unet import build_unet
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
import keras.backend as K

#  Global parameters

In [2]:
H = 256
W = 256
model_name = "model_segmentation_256px.h5"

# define some functions

In [3]:
def dice_coef(y_true, y_pred, smooth=100):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

# Directory for storing files and set hyperparameters

In [4]:
create_dir("files")

batch_size = 16
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files", model_name)
csv_path = os.path.join("files", "log.csv")


# load a Dataset

In [5]:
dataset_path = "Dataset/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)


Train: 1840 - 1840
Valid: 612 - 612
Test : 612 - 612


# Model

In [6]:
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False), # edit the (patience = 10 , restore = True) 
]


**Train the `model`**

In [7]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)


Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 0.8376 - dice_coef: 0.1624 
Epoch 1: val_loss improved from inf to 0.97269, saving model to files\model_segmentation_256px.h5


C:\Users\AI\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


115/115 [==============================] - 4869s 42s/step - loss: 0.8376 - dice_coef: 0.1624 - val_loss: 0.9727 - val_dice_coef: 0.0273 - lr: 1.0000e-04
Epoch 2/100
115/115 [==============================] - ETA: 0s - loss: 0.6956 - dice_coef: 0.3044 
Epoch 2: val_loss did not improve from 0.97269
115/115 [==============================] - 4842s 42s/step - loss: 0.6956 - dice_coef: 0.3044 - val_loss: 0.9842 - val_dice_coef: 0.0158 - lr: 1.0000e-04
Epoch 3/100
115/115 [==============================] - ETA: 0s - loss: 0.5975 - dice_coef: 0.4025 
Epoch 3: val_loss improved from 0.97269 to 0.96293, saving model to files\model_segmentation_256px.h5
115/115 [==============================] - 4848s 42s/step - loss: 0.5975 - dice_coef: 0.4025 - val_loss: 0.9629 - val_dice_coef: 0.0369 - lr: 1.0000e-04
Epoch 4/100
115/115 [==============================] - ETA: 0s - loss: 0.5204 - dice_coef: 0.4796 
Epoch 4: val_loss improved from 0.96293 to 0.78570, saving model to files\model_segmentation_25

115/115 [==============================] - ETA: 0s - loss: 0.0941 - dice_coef: 0.9059 
Epoch 26: val_loss did not improve from 0.19363
115/115 [==============================] - 4869s 42s/step - loss: 0.0941 - dice_coef: 0.9059 - val_loss: 0.2175 - val_dice_coef: 0.7818 - lr: 1.0000e-04
Epoch 27/100
115/115 [==============================] - ETA: 0s - loss: 0.0942 - dice_coef: 0.9058 
Epoch 27: val_loss did not improve from 0.19363
115/115 [==============================] - 4868s 42s/step - loss: 0.0942 - dice_coef: 0.9058 - val_loss: 0.1948 - val_dice_coef: 0.8034 - lr: 1.0000e-04
Epoch 28/100
115/115 [==============================] - ETA: 0s - loss: 0.0877 - dice_coef: 0.9123 
Epoch 28: val_loss did not improve from 0.19363

Epoch 28: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
115/115 [==============================] - 4863s 42s/step - loss: 0.0877 - dice_coef: 0.9123 - val_loss: 0.1953 - val_dice_coef: 0.8014 - lr: 1.0000e-04
Epoch 29/100
115/115 [==========

#  Directory for Results files

In [8]:
create_dir("results")

# load model after training

In [9]:
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
    model = tf.keras.models.load_model(os.path.join("files", model_name))

# Prediction and Evaluation

In [10]:
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE.append([name, f1_value, jac_value, recall_value, precision_value])

100%|████████████████████████████████| 612/612 [08:38<00:00,  1.18it/s]


In [11]:
""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"F1: {score[0]:0.5f}")
print(f"Jaccard: {score[1]:0.5f}")
print(f"Recall: {score[2]:0.5f}")
print(f"Precision: {score[3]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")

F1: 0.76724
Jaccard: 0.67986
Recall: 0.76924
Precision: 0.81618
